In [1]:
"""
Function to compute the execution time.
"""
from datetime import timedelta
import time
def get_time(f, kwargs={}):
    times = []
    result = None
    for i in range(5):
        print("measure",i+1)
        t = time.time()
        result = f(**kwargs)
        times.append(time.time()-t)
        
    print("all times:", times)
    a = min(times)
    b = max(times)
    print("times' outliers:",a,"&",b)
    t = (sum(times)-a-b)/3
    print("benchmark time",str(t)+"s","or", timedelta(seconds=t))
    return result

In [2]:
import skmob
import pandas as pd
import folium
from skmob.preprocessing import compression, filtering, detection

In [3]:
DATASET_SIZE = "sample"

# Ships trajectories

In [4]:
df = pd.read_csv('datasets/ships_'+DATASET_SIZE+'.csv')

In [5]:
# load trajectory in TrajDataframe
trajectory_df = get_time(skmob.TrajDataFrame,{"data":df, "latitude":'Latitude', "longitude":'Longitude', "trajectory_id":'id', "datetime":'Timestamp'})
trajectory_df

measure 1
measure 2
measure 3
measure 4
measure 5
all times: [0.25211524963378906, 0.026514053344726562, 0.009119987487792969, 0.008212566375732422, 0.007658958435058594]
times' outliers: 0.007658958435058594 & 0.25211524963378906
benchmark time 0.014615535736083984s or 0:00:00.014616


,lng,lat,datetime,tid,SOG,ROT,Heading,Navigational status,Name
0,8.020880,55.965527,2022-10-20 00:00:00,219015579,6.3,0.0,3.0,Restricted maneuverability,NaN
1,8.020452,54.359028,2022-10-20 00:00:01,218768000,5.6,0.0,352.0,Under way sailing,NaN
2,8.020422,54.359095,2022-10-20 00:00:04,218768000,5.5,0.0,352.0,Under way sailing,NaN
3,8.020350,54.359248,2022-10-20 00:00:11,218768000,5.5,0.0,352.0,Under way sailing,NaN
4,8.020295,54.359343,2022-10-20 00:00:14,218768000,5.5,NaN,351.0,Under way sailing,NaN
...,...,...,...,...,...,...,...,...,...
44395,7.266410,55.362103,2022-10-20 23:59:47,218768000,4.1,0.0,243.0,Under way sailing,ALEX VON HUMBOLDT 2
44396,7.266272,55.362083,2022-10-20 23:59:50,218768000,4.2,NaN,242.0,Under way sailing,ALEX VON HUMBOLDT 2
44397,9.706992,57.465592,2022-10-20 23:59:54,219015579,3.7,NaN,43.0,Restricted maneuverability,LOKE R
44398,7.266143,55.362068,2022-10-20 23:59:54,218768000,4.2,NaN,242.0,Under way sailing,ALEX VON HUMBOLDT 2


### Outlier detection

In [50]:
# Outlier detection and trajectory cleaning
cleaned_df = get_time(filtering.filter,{"tdf":trajectory_df,"max_speed_kmh":70})
cleaned_df

measure 1
measure 2
measure 3
measure 4
measure 5
all times: [0.3363206386566162, 0.3262207508087158, 0.3213815689086914, 0.27286624908447266, 0.3181486129760742]
times' outliers: 0.27286624908447266 & 0.3363206386566162
benchmark time 0.32191697756449383s or 0:00:00.321917


,tid,datetime,lat,lng
0,8,2020-06-01 08:56:08.148000+00:00,50.861073,4.465373
1,8,2020-06-01 08:56:09.648000+00:00,50.861100,4.465386
2,8,2020-06-01 08:56:10.398000+00:00,50.861127,4.465398
3,8,2020-06-01 08:56:10.797745+00:00,50.861146,4.465406
4,8,2020-06-01 08:56:24.229685+00:00,50.861756,4.465684
...,...,...,...,...
35887,141,2020-06-04 17:30:39.130722+00:00,50.860265,4.493515
35888,141,2020-06-04 17:30:40.435295+00:00,50.860265,4.493515
35889,141,2020-06-04 17:30:41.935295+00:00,50.860240,4.493535
35890,141,2020-06-04 17:30:42.685295+00:00,50.860215,4.493556


In [51]:
print("nombre precedent de points:", len(trajectory_df))
print("nombre de points retirés:", len(trajectory_df)-len(cleaned_df))
print("nombre actuel de points:", len(cleaned_df))

nombre precedent de points: 40323
nombre de points retirés: 4431
nombre actuel de points: 35892


In [35]:
cleaned_df.to_csv("original-skmob-cars.csv", index=False)

In [42]:
outliers = trajectory_df.merge(cleaned_df, how='left', on=["lat","lng"],indicator = True)
outliers = outliers[outliers["_merge"] == "left_only"]
outliers

,tid_x,datetime_x,lat,lng,tid_y,datetime_y,_merge
18,219015579,2022-10-20 00:00:55,55.967012,8.021040,NaN,NaT,left_only
45,219015579,2022-10-20 00:01:43,55.968303,8.021173,NaN,NaT,left_only
52,219015579,2022-10-20 00:02:03,55.968848,8.021230,NaN,NaT,left_only
70,219015579,2022-10-20 00:02:57,55.970302,8.021407,NaN,NaT,left_only
72,219015579,2022-10-20 00:03:03,55.970435,8.021418,NaN,NaT,left_only
...,...,...,...,...,...,...,...
45702,218768000,2022-10-20 21:34:04,55.381135,7.624802,NaN,NaT,left_only
45808,218768000,2022-10-20 21:40:04,55.381777,7.605148,NaN,NaT,left_only
45916,218768000,2022-10-20 21:46:04,55.381838,7.585695,NaN,NaT,left_only
46022,218768000,2022-10-20 21:52:04,55.382097,7.566792,NaN,NaT,left_only


# Cars trajectories

In [43]:
df = pd.read_csv('datasets/cars_'+DATASET_SIZE+'.csv')

In [44]:
# load trajectory in TrajDataframe
trajectory_df = skmob.TrajDataFrame(data=df, latitude='Latitude', longitude='Longitude', trajectory_id='id', datetime='Timestamp')

In [29]:
trajectory_df.to_csv("original-skmob-ships.csv")

### Stop detection 

In [22]:
# Stop point detection
stops = get_time(detection.stay_locations,{"tdf":trajectory_df,"minutes_for_a_stop":30,"spatial_radius_km":0.02})

measure 1
measure 2
measure 3
measure 4
measure 5
all times: [0.37206554412841797, 0.2598581314086914, 0.30888891220092773, 0.2620267868041992, 0.2627434730529785]
times' outliers: 0.2598581314086914 & 0.37206554412841797
benchmark time 0.27788639068603516s or 0:00:00.277886


In [23]:
print("nombre de stops detectés:",len(stops))

nombre de stops detectés: 18


### Compression

In [30]:
# Compression of the trajectory
compressed_df = get_time(compression.compress,{"tdf":trajectory_df,"spatial_radius_km":0.20})

measure 1
measure 2
measure 3
measure 4
measure 5
all times: [0.23450970649719238, 0.27056121826171875, 0.22189807891845703, 0.2668771743774414, 0.22130107879638672]
times' outliers: 0.22130107879638672 & 0.27056121826171875
benchmark time 0.24109498659769693s or 0:00:00.241095


In [31]:
print("nombre de points apres compression:",len(compressed_df))

nombre de points apres compression: 2097


### Visualisation

In [14]:
m1 = get_time(trajectory_df.plot_trajectory)
m1
trajectory_df.plot_trajectory

measure 1


/home/pierre-cedric/miniconda3/envs/skmob/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,
/home/pierre-cedric/miniconda3/envs/skmob/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, specify `max_points=None`.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,
/home/pierre-cedric/miniconda3/envs/skmob/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify

measure 2
measure 3
measure 4
measure 5
all times: [0.23577308654785156, 0.021298646926879883, 0.022414207458496094, 0.022164344787597656, 0.02069258689880371]
times' outliers: 0.02069258689880371 & 0.23577308654785156
benchmark time 0.02195906639099121s or 0:00:00.021959


/home/pierre-cedric/miniconda3/envs/skmob/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,
/home/pierre-cedric/miniconda3/envs/skmob/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, specify `max_points=None`.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,
/home/pierre-cedric/miniconda3/envs/skmob/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify